# Debug #

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
n_folds = 5 # this sets global setting of which how many bootstraps to use
n_hold_out = 1
importance_type = "split"

life_history = ["lifespan", "mass_kg", "mtGC", "metabolic_rate", "temperature", "gestation_days"]
debug_local = True #to use local version
trait = "mass_kg"

params = {"bagging_fraction": 0.9522534844058304,
                  "boosting_type": "dart",
                  "objective": "regression",
                  "feature_fraction": 0.42236910941558053,
                  "lambda_l1": 0.020847266580277746,
                  "lambda_l2": 2.8448564854773326,
                  "learning_rate": 0.11484015430016059,
                  "max_depth": 3,
                  "max_leaves": 35,
                  "min_data_in_leaf": 9,
                  "num_iterations": 250,
                  "metrics": ["l1", "l2", "huber"]
                 }

In [2]:
from pathlib import Path
import sys
import inspect

#lgb_params["importance_type"] = importance_type

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat, Collect
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.models import Metrics
from yspecies.selection import ShapSelector
from yspecies.results import FeatureSummary

In [5]:
import optuna
from optuna import Study, Trial
from optuna import multi_objective
from optuna.multi_objective import trial
from optuna.multi_objective.study import MultiObjectiveStudy
from yspecies.tuning import MultiObjectiveResults

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

## Setting up Features to select ##

In [8]:
selection = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = trait, #column to predict
    categorical = ["tissue"],
    select_by = "shap",
    importance_type =  importance_type,
    feature_perturbation = "tree_path_dependent"
)
selection

Samples metadata,Species metadata,Genes,Predict label,not_validated species
"['tissue', 'species']",[],all,mass_kg,[]


In [9]:
loader = DataLoader(locations, selection)
selections = loader.load_life_history()
data = selections[trait]

## Setting up SHAP selection pipeline ##

### Deciding on selection parameters (which fields to include, exclude, predict)  ###

In [10]:
partition_params = PartitionParameters(n_folds, n_hold_out, 2,   42)


In [11]:
pipe =  Pipeline([
        ('extractor', DataExtractor()),
        ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
        ("partitioner", DataPartitioner()),
        ('prepare_for_selection', TupleWith(params)),
        ("shap_computation", ShapSelector())]
        )

In [12]:
res = pipe.fit_transform(data, selection)
res

2020-08-28 20:28:40.831 | INFO     | yspecies.selection:fit:78 - ===== fitting models with seed 42 =====
2020-08-28 20:28:40.831 | INFO     | yspecies.selection:fit:79 - PARAMETERS:
{'bagging_fraction': 0.9522534844058304, 'boosting_type': 'dart', 'objective': 'regression', 'feature_fraction': 0.42236910941558053, 'lambda_l1': 0.020847266580277746, 'lambda_l2': 2.8448564854773326, 'learning_rate': 0.11484015430016059, 'max_depth': 3, 'max_leaves': 35, 'min_data_in_leaf': 9, 'num_iterations': 250, 'metrics': ['l1', 'l2', 'huber']}
2020-08-28 20:28:40.850 | INFO     | yspecies.selection:fit:82 - SEED: 42 | FOLD: 0 | VALIDATION_SPECIES: ['Suricata_suricatta', 'Oryctolagus_cuniculus']
Found `num_iterations` in params. Will use it instead of argument
Early stopping is not available in dart mode


[250]	valid_0's l1: 17.8169	valid_0's l2: 757.273	valid_0's huber: 15.6606


2020-08-28 20:28:45.974 | INFO     | yspecies.selection:fit:82 - SEED: 42 | FOLD: 1 | VALIDATION_SPECIES: ['Monodelphis_domestica', 'Heterocephalus_glaber']


[250]	valid_0's l1: 13.6973	valid_0's l2: 469.715	valid_0's huber: 11.9359


2020-08-28 20:28:50.244 | INFO     | yspecies.selection:fit:82 - SEED: 42 | FOLD: 2 | VALIDATION_SPECIES: ['Callithrix_jacchus', 'Cavia_porcellus']


[250]	valid_0's l1: 16.0078	valid_0's l2: 586.886	valid_0's huber: 14.0241


2020-08-28 20:28:52.958 | INFO     | yspecies.selection:fit:82 - SEED: 42 | FOLD: 3 | VALIDATION_SPECIES: ['Otolemur_garnettii', 'Ictidomys_tridecemlineatus']


[250]	valid_0's l1: 4.83788	valid_0's l2: 125.784	valid_0's huber: 3.97368


<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th><th>hold out metrics</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>symbol</th>
      <th>shap_absolute_sum_to_mass_kg</th>
      <th>kendall_tau_to_mass_kg</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ENSG00000066923</th>
      <td>STAG3</td>
      <td>32.000</td>
      <td>0.215</td>
    </tr>
    <tr>
      <th>ENSG00000054277</th>
      <td>OPN3</td>
      <td>22.750</td>
      <td>0.225</td>
    </tr>
    <tr>
      <th>ENSG00000144214</th>
      <td>LYG1</td>
      <td>22.500</td>
      <td>0.287</td>
    </tr>
    <tr>
      <th>ENSG00000089063</th>
      <td>TMEM230</td>
      <td>20.250</td>
      <td>-0.118</td>
    </tr>
    <tr>
      <th>ENSG00000164304</th>
      <td>CAGE1</td>
      <td>17.500</td>
      <td>-0.366</td>
    </tr>
    <tr>
      <th>ENSG00000127952</th>
      <td>STYXL1</td>
      <td>8.750</td>
      <td>0.304</td>
    </tr>
    <tr>
      <th>ENSG00000175806</th>
      <td>MSRA</td>
      <td>8.250</td>
      <td>-0.216</td>
    </tr>
    <tr>
      <th>ENSG00000075702</th>
      <td>WDR62</td>
      <td>7.000</td>
      <td>-0.103</td>
    </tr>
    <tr>
      <th>ENSG00000275111</th>
      <td>ZNF2</td>
      <td>1.250</td>
      <td>0.028</td>
    </tr>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>huber</th>
      <th>validation_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.869</td>
      <td>17.817</td>
      <td>757.273</td>
      <td>15.661</td>
      <td>[Suricata_suricatta, Oryctolagus_cuniculus]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.869</td>
      <td>13.697</td>
      <td>469.715</td>
      <td>11.936</td>
      <td>[Monodelphis_domestica, Heterocephalus_glaber]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.769</td>
      <td>16.008</td>
      <td>586.886</td>
      <td>14.024</td>
      <td>[Callithrix_jacchus, Cavia_porcellus]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.965</td>
      <td>4.838</td>
      <td>125.784</td>
      <td>3.974</td>
      <td>[Otolemur_garnettii, Ictidomys_tridecemlineatus]</td>
    </tr>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>huber</th>
      <th>hold_out_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.863</td>
      <td>12.121</td>
      <td>487.992</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.888</td>
      <td>12.691</td>
      <td>397.564</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.809</td>
      <td>17.350</td>
      <td>680.837</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.934</td>


In [17]:
res.selected.index

Index(['ENSG00000066923', 'ENSG00000054277', 'ENSG00000144214',
       'ENSG00000089063', 'ENSG00000164304', 'ENSG00000127952',
       'ENSG00000175806', 'ENSG00000075702', 'ENSG00000275111'],
      dtype='object')

In [18]:
empty_selected = pd.DataFrame(columns=["symbol","shap_absolute_sum_to_mass_kg", "kendall_tau_to_mass_kg"])
empty_selected.index.name = "ensembl_id"
empty_selected

,symbol,shap_absolute_sum_to_mass_kg,kendall_tau_to_mass_kg
ensembl_id,,,


In [19]:
from yspecies.results import FeatureSummary


In [23]:
e = replace(res,selected=empty_selected)
e

<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th><th>hold out metrics</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>symbol</th>
      <th>shap_absolute_sum_to_mass_kg</th>
      <th>kendall_tau_to_mass_kg</th>
    </tr>
    <tr>
      <th>ensembl_id</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>huber</th>
      <th>validation_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.869</td>
      <td>17.817</td>
      <td>757.273</td>
      <td>15.661</td>
      <td>[Suricata_suricatta, Oryctolagus_cuniculus]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.869</td>
      <td>13.697</td>
      <td>469.715</td>
      <td>11.936</td>
      <td>[Monodelphis_domestica, Heterocephalus_glaber]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.769</td>
      <td>16.008</td>
      <td>586.886</td>
      <td>14.024</td>
      <td>[Callithrix_jacchus, Cavia_porcellus]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.965</td>
      <td>4.838</td>
      <td>125.784</td>
      <td>3.974</td>
      <td>[Otolemur_garnettii, Ictidomys_tridecemlineatus]</td>
    </tr>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>huber</th>
      <th>hold_out_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.863</td>
      <td>12.121</td>
      <td>487.992</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.888</td>
      <td>12.691</td>
      <td>397.564</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.809</td>
      <td>17.350</td>
      <td>680.837</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.934</td>
      <td>8.925</td>
      <td>235.935</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
  </tbody>
</table>
</div></th></tr></table>

In [28]:
f = FeatureSummary([res, e])
f

FeatureSummary(results=[<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th><th>hold out metrics</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>symbol</th>
      <th>shap_absolute_sum_to_mass_kg</th>
      <th>kendall_tau_to_mass_kg</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ENSG00000066923</th>
      <td>STAG3</td>
      <td>32.000</td>
      <td>0.215</td>
    </tr>
    <tr>
      <th>ENSG00000054277</th>
      <td>OPN3</td>
      <td>22.750</td>
      <td>0.225</td>
    </tr>
    <tr>
      <th>ENSG00000144214</th>
      <td>LYG1</td>
      <td>22.500</td>
      <td>0.287</td>
    </tr>
    <tr>
      <th>ENSG00000089063</th>
      <td>TMEM230</td>
      <td>20.250</td>
      <td>-0.118</td>
    </tr>
    <tr>
      <th>ENSG00000164304</th>
      <td>CAGE1</td>
      <td>17.500</td>
      <td>-0.366</td>
    </tr>
    <tr>
      <th>ENSG00000127952</th>
      <td>STYXL1</td>
      <td>8.750</td>
      <td>0.304</td>
    </tr>
    <tr>
      <th>ENSG00000175806</th>
      <td>MSRA</td>
      <td>8.250</td>
      <td>-0.216</td>
    </tr>
    <tr>
      <th>ENSG00000075702</th>
      <td>WDR62</td>
      <td>7.000</td>
      <td>-0.103</td>
    </tr>
    <tr>
      <th>ENSG00000275111</th>
      <td>ZNF2</td>
      <td>1.250</td>
      <td>0.028</td>
    </tr>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>huber</th>
      <th>validation_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.869</td>
      <td>17.817</td>
      <td>757.273</td>
      <td>15.661</td>
      <td>[Suricata_suricatta, Oryctolagus_cuniculus]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.869</td>
      <td>13.697</td>
      <td>469.715</td>
      <td>11.936</td>
      <td>[Monodelphis_domestica, Heterocephalus_glaber]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.769</td>
      <td>16.008</td>
      <td>586.886</td>
      <td>14.024</td>
      <td>[Callithrix_jacchus, Cavia_porcellus]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>0.965</td>
      <td>4.838</td>
      <td>125.784</td>
      <td>3.974</td>
      <td>[Otolemur_garnettii, Ictidomys_tridecemlineatus]</td>
    </tr>
  </tbody>
</table>
</div></th><th><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>R^2</th>
      <th>MAE</th>
      <th>MSE</th>
      <th>huber</th>
      <th>hold_out_species</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>0.863</td>
      <td>12.121</td>
      <td>487.992</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>0.888</td>
      <td>12.691</td>
      <td>397.564</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>0.809</td>
      <td>17.350</td>
      <td>680.837</td>
      <td>nan</td>
      <td>[Phascolarctos_cinereus, Aotus_nancymaae]</td>
    </tr>
    <tr>
      <th>3</t